In [1]:
!pip install -qU datasets==2.12.0 qdrant-client==1.2.0 sentence-transformers==2.2.2 torch==2.0.1

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 474.6/474.6 kB 6.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.9/129.9 kB 6.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 kB 4.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 619.9/619.9 MB 2.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.5/110.5 kB 15.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 19.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.7/2.7 MB 73.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.0/75.0 kB 8.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.8/143.8 kB 16.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 70.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.0/21.0 MB 56.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 849.3/849.3 kB 60.9

In [2]:
import torch
from datasets import load_dataset
from transformers import pipeline
from sentence_transformers import SentenceTransformer
from qdrant_client import QdrantClient
from qdrant_client.http import models
from tqdm.auto import tqdm
from typing import List
import pandas as pd

In [46]:
df = pd.read_csv("bigBasketProducts.csv")
# df = load_dataset("duorc", "ParaphraseRC", split="train").to_pandas()
# df = df[["product", "description"]]  # select only title and plot column
# print(f"Before removing duplicates: {len(df)}")
df = df.drop(['index'],axis=1)
# df = df.drop_duplicates(subset="plot")  # drop rows containing duplicate plot passages, if any
print(f"Products: {len(df)}")
df.head()

Products: 27555


,product,category,sub_category,brand,sale_price,market_price,type,rating,description
0,Garlic Oil - Vegetarian Capsule 500 mg,Beauty & Hygiene,Hair Care,Sri Sri Ayurveda,220.0,220.0,Hair Oil & Serum,4.1,This Product contains Garlic Oil that is known...
1,Water Bottle - Orange,"Kitchen, Garden & Pets",Storage & Accessories,Mastercook,180.0,180.0,Water & Fridge Bottles,2.3,"Each product is microwave safe (without lid), ..."
2,"Brass Angle Deep - Plain, No.2",Cleaning & Household,Pooja Needs,Trm,119.0,250.0,Lamp & Lamp Oil,3.4,"A perfect gift for all occasions, be it your m..."
3,Cereal Flip Lid Container/Storage Jar - Assort...,Cleaning & Household,Bins & Bathroom Ware,Nakoda,149.0,176.0,"Laundry, Storage Baskets",3.7,Multipurpose container with an attractive desi...
4,Creme Soft Soap - For Hands & Body,Beauty & Hygiene,Bath & Hand Wash,Nivea,162.0,162.0,Bathing Bars & Soaps,4.4,Nivea Creme Soft Soap gives your skin the best...


In [4]:
client = QdrantClient(":memory:")

In [5]:
collection_name = "chaabi-llm2"

collections = client.get_collections()
print(collections)

# only create collection if it doesn't exist
if collection_name not in [c.name for c in collections.collections]:
    client.recreate_collection(
        collection_name=collection_name,
        vectors_config=models.VectorParams(
            size=384,
            distance=models.Distance.COSINE,
        ),
    )
collections = client.get_collections()
print(collections)

collections=[]
collections=[CollectionDescription(name='chaabi-llm2')]


In [6]:

device = "cuda" if torch.cuda.is_available() else "cpu"

# load the retriever model from huggingface model hub
retriever = SentenceTransformer("multi-qa-MiniLM-L6-cos-v1", device=device)
retriever

.gitattributes:   0%|          | 0.00/737 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/11.5k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/612 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/383 [00:00<?, ?B/s]

train_script.py:   0%|          | 0.00/13.8k [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

SentenceTransformer(
  (0): Transformer({'max_seq_length': 512, 'do_lower_case': False}) with Transformer model: BertModel 
  (1): Pooling({'word_embedding_dimension': 384, 'pooling_mode_cls_token': False, 'pooling_mode_mean_tokens': True, 'pooling_mode_max_tokens': False, 'pooling_mode_mean_sqrt_len_tokens': False})
  (2): Normalize()
)

In [7]:
%%time

batch_size = 512  # specify batch size according to your RAM and compute, higher batch size = more RAM usage

for index in tqdm(range(0, len(df), batch_size)):
    i_end = min(index + batch_size, len(df))  # find end of batch
    batch = df.iloc[index:i_end]  # extract batch
    text_columns = batch[["product", "description","sub_category","category","brand"]].astype(str).agg(' | '.join, axis=1).tolist()
    # Generate embeddings for the batch
    emb = retriever.encode(text_columns).tolist()
    # emb = retriever.encode(batch["description"].tolist()).tolist()  # generate embeddings for batch
    meta = batch.to_dict(orient="records")  # get metadata
    ids = list(range(index, i_end))  # create unique IDs

    # upsert to qdrant
    client.upsert(
        collection_name=collection_name,
        points=models.Batch(ids=ids, vectors=emb, payloads=meta),
    )

collection_vector_count = client.get_collection(collection_name=collection_name).vectors_count
print(f"Vector count in collection: {collection_vector_count}")
assert collection_vector_count == len(df)


  0%|          | 0/54 [00:00<?, ?it/s]

Vector count in collection: 27555
CPU times: user 2min 2s, sys: 1.77 s, total: 2min 3s
Wall time: 2min 5s


In [8]:
model_name = "bert-large-uncased-whole-word-masking-finetuned-squad"

# load the reader model into a question-answering pipeline
reader = pipeline("question-answering", model=model_name, tokenizer=model_name)
print(reader.model, reader)

config.json:   0%|          | 0.00/443 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.34G [00:00<?, ?B/s]

Some weights of the model checkpoint at bert-large-uncased-whole-word-masking-finetuned-squad were not used when initializing BertForQuestionAnswering: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight']
- This IS expected if you are initializing BertForQuestionAnswering from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForQuestionAnswering from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


tokenizer_config.json:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

BertForQuestionAnswering(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 1024, padding_idx=0)
      (position_embeddings): Embedding(512, 1024)
      (token_type_embeddings): Embedding(2, 1024)
      (LayerNorm): LayerNorm((1024,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-23): 24 x BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=1024, out_features=1024, bias=True)
              (key): Linear(in_features=1024, out_features=1024, bias=True)
              (value): Linear(in_features=1024, out_features=1024, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=1024, out_features=1024, bias=True)
              (LayerNorm): LayerNorm((1024,), ep

In [9]:
def get_relevant_plot(question: str, top_k: int) -> List[str]:
    try:
        encoded_query = retriever.encode(question).tolist()  # generate embeddings for the question

#         payload_filter = {
#             "column_name": "desired_value"
#         }

        result = client.search(
            collection_name=collection_name,
            query_vector=encoded_query,
            limit=top_k,
        )  # search qdrant collection for context passage with the answer
#         print(result)
        context = [
            [x.payload["product"], x.payload["description"]] for x in result
        ]  # extract title and payload from result
        return context

    except Exception as e:
        print({e})

In [29]:
def extract_answer(question: str, context: List[str]):
    results = []
    for c in context:
        # feed the reader the question and contexts to extract answers
        answer = reader(question=question, context=c[1])

        # add the context to answer dict for printing both together, we print only first 500 characters of plot
        answer["product"] = c[0]
        results.append(answer)

    # sort the result based on the score from reader model
    sorted_result = sorted(results, key=lambda x: x["score"], reverse=True)
    print(
          "  Answer: ",
          sorted_result[0]["answer"],
          "\n  Title: ",
          sorted_result[0]["product"],
          "\n  score: ",
          sorted_result[0]["score"],
      )
    # for i in range(len(sorted_result)):
    #     print(f"{i+1}", end=" ")
    #     print(
    #         "Answer: ",
    #         sorted_result[i]["answer"],
    #         "\n  Title: ",
    #         sorted_result[i]["product"],
    #         "\n  score: ",
    #         sorted_result[i]["score"],
    #     )


question = "which product is used for dry skin"
context = get_relevant_plot(question, top_k=1)
# context


In [37]:
question = "which product is produced by Nutrashil brand"
context = get_relevant_plot(question, top_k=5)
extract_answer(question, context)


  Answer:  Nutriorg Jamun Honey 
  Title:  Certified Organic Jamun Honey 
  score:  0.7772384881973267


In [36]:
question = "Which product contains garlic oil"
context = get_relevant_plot(question, top_k=5)
extract_answer(question, context)


  Answer:  Colavita Extra Virgin Olive Oil 
  Title:  Garlicolio - Garlic Seasoning Made With Extra Virgin Olive Oil 
  score:  0.5382457971572876


In [35]:
question = "Which product has is made of Happydent Wave Xylitol"
context = get_relevant_plot(question, top_k=5)
extract_answer(question, context)


  Answer:  bubble gum 
  Title:  Xylitol Sugarfree Fruits Flavour - Bubble Gum 
  score:  0.2976195216178894


In [40]:
question = "What is called as Indian superfruit and also an Indian gooseberry?"
context = get_relevant_plot(question, top_k=10)
extract_answer(question, context)


  Answer:  amla 
  Title:  Amla 
  score:  0.9943625330924988


In [38]:
question = "What wax is a sculpting product in the Schwarzkopf line? "
context = get_relevant_plot(question, top_k=5)
extract_answer(question, context)


  Answer:  Taft Hair Wax 
  Title:  Taft Shine Gel Wax 
  score:  0.8936730027198792


In [45]:
question = "What is the size of designer glass bowl? "
context = get_relevant_plot(question, top_k=5)
extract_answer(question, context)


  Answer:  280 ML 
  Title:  Designer Glass Bowl 
  score:  0.7578027248382568


In [57]:
question = "What is the flavor of Tetley Black Tea"
context = get_relevant_plot(question, top_k=5)
extract_answer(question, context)


  Answer:  natural masala flavour 
  Title:  Masala Tea 
  score:  0.4799739420413971


In [58]:
question = "Which is ideal for replacing freshly whipped cream"
context = get_relevant_plot(question, top_k=5)
extract_answer(question, context)


  Answer:  It is a free-flowing cream chocolate flavoured powder 
  Title:  Whipped Cream Powder - Chocolate 
  score:  0.4593959152698517


In [41]:
import pickle

# Save to file in the current working directory
pkl_filename = "model.pkl"
with open(pkl_filename, 'wb') as file:
    pickle.dump(reader.model, file)

# Load from file
with open(pkl_filename, 'rb') as file:
    pickle_model = pickle.load(file)

In [26]:
client.delete_collection(collection_name=collection_name)

True